In [8]:
import json
from typing import Annotated

import nest_asyncio
import requests
import uvicorn
from fastapi import FastAPI, Query,Path,Body
from pydantic import BaseModel
from typing import Any


In [ ]:
app = FastAPI()

class UserInput(BaseModel):
    username: str
    password: str  # Sensitive field

class UserOutput(BaseModel):
    username: str


@app.post("/notsecure/")
async def create_user(user: UserInput):
    return user  


@app.post("/secure/", response_model=UserOutput)
async def create_user(user: UserInput):
    return user
    

if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app, port=4444)

INFO:     Started server process [5080]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4444 (Press CTRL+C to quit)


INFO:     127.0.0.1:55144 - "POST /notsecure/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:55145 - "POST /secure/ HTTP/1.1" 200 OK
